In [1]:
from relationGraph import Relation, RelationGraph, MatrixOfRelationGraph
from autoencoder import seedy, AutoEncoder
from main import test_build_relation_graph_with_symertic_data, test_convert_graph_to_2D_matrix, test_get_matix_for_autoencoder, test_autoencoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from base import load_source
from base2 import load_dicty
from os.path import join

import utilityFunctions as uf
import multiprocessing
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
# OLD - not used any more
# gene = 'Gene'
# go_term = 'GO term'
# exprc = 'Experimental condition'

# data, rn, cn = load_source(join('dicty', 'dicty.gene_annnotations.csv.gz'))
# data = uf.normalization(data)
# ann = Relation(data=data, x_name=gene, y_name=go_term, name='ann',
#                x_metadata=rn, y_metadata=cn)
# print(np.min(data))
# print(np.max(data))
# print()

# data, rn, cn = load_source(join('dicty', 'dicty.gene_expression.csv.gz'))
# expr = Relation(data=data, x_name=gene, y_name=exprc, name='expr',
#                 x_metadata=rn, y_metadata=cn)
# expr.matrix = np.log(np.maximum(expr.matrix, np.finfo(np.float).eps))
# expr.matrix = uf.normalization(expr.matrix)
# print(np.min(expr.matrix))
# print(np.max(expr.matrix))
# print()

# data, rn, cn = load_source(join('dicty', 'dicty.ppi.csv.gz'))
# data = uf.normalization(data)
# ppi = Relation(data=data, x_name=gene, y_name=gene, name='ppi',
#                x_metadata=rn, y_metadata=cn)
# print(np.min(data))
# print(np.max(data))

# ann_t = ann.transpose()
# expr_t = expr.transpose()

# relationGraph = RelationGraph()
# relationGraph.add_relations([ann, expr, ppi, ann_t, expr_t])
# relationGraph.display_objects()
# graph = relationGraph

0.0
1.0

0.0
1.0

0.0
1.0
-------------RelationGraph-------------
Gene	1219
3	ann-(1219, 116), expr-(1219, 282), ppi-(1219, 1219)
2	ann_T-(116, 1219), expr_T-(282, 1219)
GO term	116
1	ann_T-(116, 1219)
1	ann-(1219, 116)
Experimental condition	282
1	expr_T-(282, 1219)
1	expr-(1219, 282)



In [2]:
graph1 = load_dicty('clustering', 2)
# graph1 = load_dicty(None, 2)

0.0
1.0

0.0
1.0

0.0
1.0
-------------RelationGraph-------------
Gene	1024
3	ann-(1024, 128), expr-(1024, 256), ppi-(1024, 1024)
0	
GO term	128
0	
1	ann-(1024, 128)
Experimental condition	256
0	
1	expr-(1024, 256)



In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# display data
already_used = set()
list_data = []
for obj in graph1.objects.values():
    for relation in obj.relation_x:
        if relation.name not in already_used:
            list_data.append(relation.matrix)
            already_used.add(relation.name)
    
    for relation in obj.relation_y:
        if relation.name not in already_used:
            list_data.append(relation.matrix)
            already_used.add(relation.name)
        

color_pack = 'viridis'

for data in list_data:
#     fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(25, 6))
    plt.matshow(data, cmap=plt.cm.Blues)

    plt.show()

    

<Figure size 400x1600 with 1 Axes>

<Figure size 400x1600 with 1 Axes>

<Figure size 480x480 with 1 Axes>

In [4]:
# def sample_generator4(matrix, num_of_samples=1, dismissed=0.2):
#     # input  => sample_generator4([1219, 116], 1000, 0.8)
#     # output => [1000, 1219, 116, 1]
#     x_size, y_size = matrix.shape
#     data = np.zeros((0, x_size, y_size, 1))
#     print('Random seed: '  + str(np.random.randint(y_size)))
    
#     while num_of_samples > 0:
#         tmp_matrix = np.copy(matrix)
        
#         tmp_matrix = tmp_matrix.flatten()
#         nonzeros = np.nonzero(tmp_matrix)[0]
#         dismiss = np.ceil(len(nonzeros) * dismissed)
        
#         if dismissed < 1 and np.ceil(len(nonzeros) * dismissed) == len(nonzeros):
#             dismiss = np.ceil(len(nonzeros) * dismissed) - 1
        
#         idx = np.random.choice(len(nonzeros), int(dismiss), replace=False)
#         mask = np.isin(nonzeros, idx)
#         remove = nonzeros[mask]
#         tmp_matrix[remove] = 0 
            
#         data = np.r_[data, tmp_matrix.reshape(1, x_size, y_size, 1)]                
#         num_of_samples -= 1

#     return data


def mp_worker(arr):
#     new_data = uf.sample_generator3(data, num_of_samples=100, density=0.7, 1)
    np.random.seed(arr[3])
    new_data = uf.sample_generator4(arr[0], num_of_samples=arr[1], dismissed=arr[2])
    return new_data


def data_generator_2(data, n_samples, pools, dismissed, filename):
    batch_size = 1
    p = multiprocessing.Pool(pools)
    gen_samples = np.zeros((0, data.shape[0] * data.shape[1]))
    iterations = int(np.round((n_samples -1)/batch_size)) + 1
    
    params = [[data, batch_size, dismissed, i] for i in range(iterations)]
    
    i = 1
    f = uf.my_savez(filename)
    for result in p.imap(mp_worker, params):
        print('samples: ' + str(i * batch_size))
        i+=1
        f.savez(result)
    f.close()


n_samples=3000
pools=6
dismissed= .09
folder = '/home/lpodgorsek/data/cnn/dicty/_09/'

graph = graph1
# graph = graph2
for data in list_data:
    if data.shape[1] == 128:
        data_generator_2(data, n_samples, pools, dismissed, folder + 'ann.npz')
    if data.shape[1] == 256:
        data_generator_2(data, n_samples, pools, dismissed, folder + 'expr.npz')
    if data.shape[1] == 1024:
        data_generator_2(data, n_samples, pools, dismissed, folder + 'ppi.npz')

Random seed: 44
Random seed: 37
Random seed: 40
Random seed: 106
Random seed: 122
Random seed: 10
Random seed: 47
Random seed: 99
Random seed: 67
Random seed: 126
Random seed: 25
Random seed: 9
Random seed: 75
Random seed: 82
Random seed: 107
Random seed: 72
Random seed: 111
Random seed: 41
Random seed: 42
Random seed: 93
Random seed: 73
Random seed: 99
Random seed: 117
Random seed: 83
Random seed: 34
Random seed: 4
Random seed: 53
Random seed: 19
Random seed: 1
Random seed: 85
Random seed: 37
Random seed: 82
Random seed: 87
Random seed: 20
Random seed: 33
Random seed: 73
Random seed: 5
Random seed: 15
Random seed: 33
Random seed: 9
Random seed: 70
Random seed: 64
Random seed: 68
Random seed: 102
Random seed: 20
Random seed: 75
Random seed: 61
Random seed: 7
Random seed: 0
Random seed: 42
Random seed: 48
Random seed: 126
Random seed: 117
Random seed: 25
Random seed: 111
Random seed: 77
Random seed: 85
Random seed: 79
Random seed: 35
Random seed: 49
Random seed: 77
Random seed: 99
Rando

Random seed: 112
Random seed: 4
Random seed: 121
Random seed: 7
Random seed: 37
Random seed: 26
Random seed: 109
Random seed: 12
Random seed: 82
Random seed: 25
Random seed: 12
Random seed: 57
Random seed: 108
Random seed: 105
Random seed: 23
Random seed: 90
Random seed: 87
Random seed: 19
Random seed: 61
Random seed: 7
Random seed: 9
Random seed: 52
Random seed: 112
Random seed: 37
Random seed: 122
Random seed: 56
Random seed: 117
Random seed: 67
Random seed: 20
Random seed: 73
Random seed: 127
Random seed: 70
Random seed: 33
Random seed: 105
Random seed: 24
Random seed: 44
Random seed: 18
Random seed: 20
Random seed: 90
Random seed: 122
Random seed: 10
Random seed: 121
Random seed: 90
Random seed: 12
Random seed: 9
Random seed: 65
Random seed: 32
Random seed: 57
Random seed: 68
Random seed: 109
Random seed: 101
Random seed: 28
Random seed: 108
Random seed: 55
Random seed: 12
Random seed: 83
Random seed: 6
Random seed: 119
Random seed: 121
Random seed: 29
Random seed: 61
Random seed: 

Random seed: 89
Random seed: 53
Random seed: 72
Random seed: 2
Random seed: 11
Random seed: 55
Random seed: 93
Random seed: 35
Random seed: 30
Random seed: 58
Random seed: 56
Random seed: 56
Random seed: 31
Random seed: 32
Random seed: 127
Random seed: 16
Random seed: 125
Random seed: 109
Random seed: 122
Random seed: 34
Random seed: 65
Random seed: 58
Random seed: 70
Random seed: 82
Random seed: 51
Random seed: 121
Random seed: 88
Random seed: 46
Random seed: 120
Random seed: 73
Random seed: 48
Random seed: 90
Random seed: 9
Random seed: 78
Random seed: 116
Random seed: 95
Random seed: 64
Random seed: 51
Random seed: 109
Random seed: 103
Random seed: 29
Random seed: 2
Random seed: 99
Random seed: 92
Random seed: 122
Random seed: 24
Random seed: 79
Random seed: 36
Random seed: 11
Random seed: 58
Random seed: 72
Random seed: 21
Random seed: 126
Random seed: 52
Random seed: 39
Random seed: 57
Random seed: 69
Random seed: 63
Random seed: 17
Random seed: 12
Random seed: 71
Random seed: 59


Random seed: 82
Random seed: 19
Random seed: 35
Random seed: 25
Random seed: 99
Random seed: 13
Random seed: 67
Random seed: 83
Random seed: 104
Random seed: 110
Random seed: 13
Random seed: 80
Random seed: 44
Random seed: 16
Random seed: 54
Random seed: 110
Random seed: 109
Random seed: 110
Random seed: 82
Random seed: 83
Random seed: 29
Random seed: 26
Random seed: 70
Random seed: 36
Random seed: 96
Random seed: 14
samples: 103
samples: 104
samples: 105
samples: 106
samples: 107
samples: 108
samples: 109
samples: 110
samples: 111
Random seed: 86
Random seed: 17
Random seed: 116
Random seed: 40
Random seed: 75
Random seed: 92
Random seed: 48
Random seed: 71
Random seed: 95
Random seed: 4
Random seed: 105
Random seed: 123
Random seed: 115
Random seed: 114
Random seed: 87
Random seed: 123
Random seed: 105
Random seed: 20
Random seed: 87
Random seed: 75
Random seed: 13
Random seed: 16
Random seed: 74
Random seed: 56
Random seed: 81
Random seed: 0
Random seed: 19
Random seed: 19
Random se

Random seed: 85
Random seed: 102
Random seed: 33
Random seed: 93
Random seed: 1
Random seed: 8
Random seed: 119
Random seed: 20
Random seed: 28
Random seed: 91
Random seed: 86
Random seed: 125
Random seed: 127
Random seed: 122
Random seed: 42
Random seed: 98
Random seed: 72
Random seed: 91
Random seed: 77
Random seed: 125
Random seed: 85
Random seed: 46
Random seed: 19
Random seed: 100
Random seed: 98
Random seed: 13
Random seed: 87
Random seed: 25
Random seed: 98
Random seed: 108
Random seed: 69
Random seed: 111
Random seed: 99
Random seed: 72
Random seed: 91
Random seed: 20
Random seed: 95
Random seed: 15
Random seed: 1
Random seed: 24
Random seed: 7
Random seed: 36
Random seed: 41
Random seed: 91
Random seed: 81
Random seed: 105
Random seed: 51
Random seed: 120
Random seed: 117
Random seed: 108
Random seed: 29
Random seed: 85
Random seed: 119
Random seed: 92
samples: 136
samples: 137
samples: 138
samples: 139
samples: 140
samples: 141
samples: 142
samples: 143
Random seed: 37
Random

...

Random seed: 664
Random seed: 322
Random seed: 496
Random seed: 636
Random seed: 741
Random seed: 90
Random seed: 399
Random seed: 352
Random seed: 186
Random seed: 131
Random seed: 175
samples: 772
samples: 773
samples: 774
Random seed: 83
Random seed: 90
Random seed: 503
Random seed: 134
Random seed: 733
Random seed: 589
Random seed: 535
Random seed: 217
samples: 775
samples: 776
samples: 777
Random seed: 871
Random seed: 749
Random seed: 176
Random seed: 200
Random seed: 969
Random seed: 814
Random seed: 550
Random seed: 530
samples: 778
samples: 779
Random seed: 205
Random seed: 836
Random seed: 124
Random seed: 856
Random seed: 330
Random seed: 374
Random seed: 949
Random seed: 970
Random seed: 438
Random seed: 145
Random seed: 868
samples: 780
samples: 781
samples: 782
Random seed: 299
Random seed: 284
Random seed: 247
Random seed: 577
Random seed: 412
Random seed: 170
Random seed: 753
samples: 783
samples: 784
samples: 785
Random seed: 611
Random seed: 919
Random seed: 89
Random

Random seed: 994
Random seed: 867
Random seed: 434
Random seed: 409
Random seed: 638
samples: 887
samples: 888
samples: 889
Random seed: 450
Random seed: 741
Random seed: 813
Random seed: 460
Random seed: 227
Random seed: 809
samples: 890
samples: 891
samples: 892
Random seed: 73
Random seed: 824
Random seed: 726
Random seed: 696
Random seed: 362
Random seed: 596
Random seed: 604
Random seed: 141
Random seed: 991
samples: 893
samples: 894
samples: 895
Random seed: 99
Random seed: 97
Random seed: 819
Random seed: 62
Random seed: 352
Random seed: 9
Random seed: 21
Random seed: 17
Random seed: 152
samples: 896
samples: 897
samples: 898
samples: 899
samples: 900
Random seed: 581
Random seed: 331
Random seed: 559
Random seed: 255
Random seed: 786
Random seed: 679
Random seed: 981
Random seed: 9
Random seed: 822
samples: 901
samples: 902
samples: 903
Random seed: 586
Random seed: 560
Random seed: 327
Random seed: 156
Random seed: 995
Random seed: 148
Random seed: 103
Random seed: 284
Random 

samples: 1390
samples: 1391
samples: 1392
samples: 1393
samples: 1394
samples: 1395
samples: 1396
samples: 1397
samples: 1398
samples: 1399
samples: 1400
samples: 1401
samples: 1402
samples: 1403
samples: 1404
samples: 1405
samples: 1406
samples: 1407
samples: 1408
samples: 1409
samples: 1410
samples: 1411
samples: 1412
samples: 1413
samples: 1414
samples: 1415
samples: 1416
samples: 1417
samples: 1418
samples: 1419
samples: 1420
samples: 1421
samples: 1422
samples: 1423
samples: 1424
samples: 1425
samples: 1426
samples: 1427
samples: 1428
samples: 1429
samples: 1430
samples: 1431
samples: 1432
samples: 1433
samples: 1434
samples: 1435
samples: 1436
samples: 1437
samples: 1438
samples: 1439
samples: 1440
samples: 1441
samples: 1442
samples: 1443
samples: 1444
samples: 1445
samples: 1446
samples: 1447
samples: 1448
samples: 1449
samples: 1450
samples: 1451
samples: 1452
samples: 1453
samples: 1454
samples: 1455
samples: 1456
samples: 1457
samples: 1458
samples: 1459
samples: 1460
sample

samples: 1984
samples: 1985
samples: 1986
samples: 1987
samples: 1988
samples: 1989
samples: 1990
samples: 1991
samples: 1992
samples: 1993
samples: 1994
samples: 1995
samples: 1996
samples: 1997
samples: 1998
samples: 1999
samples: 2000
samples: 2001
samples: 2002
samples: 2003
samples: 2004
samples: 2005
samples: 2006
samples: 2007
samples: 2008
samples: 2009
samples: 2010
samples: 2011
samples: 2012
samples: 2013
samples: 2014
samples: 2015
samples: 2016
samples: 2017
samples: 2018
samples: 2019
samples: 2020
samples: 2021
samples: 2022
samples: 2023
samples: 2024
samples: 2025
samples: 2026
samples: 2027
samples: 2028
samples: 2029
samples: 2030
samples: 2031
samples: 2032
samples: 2033
samples: 2034
samples: 2035
samples: 2036
samples: 2037
samples: 2038
samples: 2039
samples: 2040
samples: 2041
samples: 2042
samples: 2043
samples: 2044
samples: 2045
samples: 2046
samples: 2047
samples: 2048
samples: 2049
samples: 2050
samples: 2051
samples: 2052
samples: 2053
samples: 2054
sample

samples: 2579
samples: 2580
samples: 2581
samples: 2582
samples: 2583
samples: 2584
samples: 2585
samples: 2586
samples: 2587
samples: 2588
samples: 2589
samples: 2590
samples: 2591
samples: 2592
samples: 2593
samples: 2594
samples: 2595
samples: 2596
samples: 2597
samples: 2598
samples: 2599
samples: 2600
samples: 2601
samples: 2602
samples: 2603
samples: 2604
samples: 2605
samples: 2606
samples: 2607
samples: 2608
samples: 2609
samples: 2610
samples: 2611
samples: 2612
samples: 2613
samples: 2614
samples: 2615
samples: 2616
samples: 2617
samples: 2618
samples: 2619
samples: 2620
samples: 2621
samples: 2622
samples: 2623
samples: 2624
samples: 2625
samples: 2626
samples: 2627
samples: 2628
samples: 2629
samples: 2630
samples: 2631
samples: 2632
samples: 2633
samples: 2634
samples: 2635
samples: 2636
samples: 2637
samples: 2638
samples: 2639
samples: 2640
samples: 2641
samples: 2642
samples: 2643
samples: 2644
samples: 2645
samples: 2646
samples: 2647
samples: 2648
samples: 2649
sample

In [27]:
# def sample_generator(matrix, num_of_samples=1, density=0.8):
#     # vhod v NN je celotna matrika
#     x_size, y_size = matrix.shape
#     data = np.empty((0, x_size * y_size))
#     print('Random seed: '  + str(np.random.randint(y_size)))
    
#     while num_of_samples > 0:
#         tmp_matrix = np.copy(matrix)
#         for row in tmp_matrix:
#             indices = np.random.choice(y_size, round(y_size * density), replace=False)
#             mask = np.isin(np.arange(y_size), indices, invert=True)
#             row[mask] = 0
            
#         data = np.r_[data, tmp_matrix.reshape(1, x_size * y_size)]                
#         num_of_samples -= 1

#     return data


# mrg = MatrixOfRelationGraph(graph=graph)
# mrg.convert_to_2D_matrix()
# mrg.display_metadata_2D_matrix()

# # list_of_density = [.05, .1, .15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9, .95, .1]
# list_of_density = [.05, .1, .15, .2, .25,]
# for density in list_of_density: 
#     data = mrg.density_data(density)
#     print('Input: ' + str(data.shape))

#     # save original data
#     fn = '/data/samples/dicty/' + str(data.shape[0]) + '_org_data.npz'
#     data_package = sample_generator(data, 100, 1)
#     print('Package: ' + str(data_package.shape))
#     f = uf.my_savez(fn)
#     f.savez(data_package)
#     f.close()
    
    
#     f = np.load(fn)
#     data_x = f[f.files[0]]
#     print('Read: ' + str(data_x.shape))
#     print()

#     # generated test data 
#     cpu = 6 
#     density = .8
#     num_samples = 50000

# #     fn = '/data/samples/dicty/' + str(data.shape[0]) + '_data.npz'
# #     uf.data_generator(data, num_samples, cpu, density, fn)

# print('Finnish!!!')

-----------ann_T (116, 1219)-----------
(116, 1219)

-----------ann (1219, 116)-----------
x != y
(1335, 1335)

-----------expr_T (282, 1219)-----------
y == 0
(1617, 1335)

-----------expr (1219, 282)-----------
x == 0
(1617, 1617)

-----------ppi (1219, 1219)-----------
x == y
(1617, 1617)

-------------2D Matrix-------------
Objects: GO term: (0, (0, 115)), Experimental condition: (1, (116, 397)), Gene: (2, (398, 1616))
[0. 0. 1.]
[0. 0. 1.]
[1. 1. 1.]

GO term: 6
Experimental condition: 14
Gene: 61
Input: (81, 81)
Random seed: 66
Package: (100, 6561)
Read: (100, 6561)

GO term: 12
Experimental condition: 28
Gene: 122
Input: (162, 162)
Random seed: 54
Package: (100, 26244)
Read: (100, 26244)

GO term: 17
Experimental condition: 42
Gene: 183
Input: (242, 242)
Random seed: 220
Package: (100, 58564)
Read: (100, 58564)

GO term: 23
Experimental condition: 56
Gene: 244
Input: (323, 323)
Random seed: 287
Package: (100, 104329)
Read: (100, 104329)

GO term: 29
Experimental condition: 70
Ge